In [1]:
# Install Libraries
!pip install langdetect
!pip install pycountry
!pip install textblob
!pip install tweepy
!pip install wordcloud

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Import Libraries
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\zjw64\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
# Authentication
apiKey = "apiKey"
apiSecret = "apiSecret"
accessToken = "accessToken" 
accessTokenSecret = "accessTokenSecret"
auth = tweepy.OAuthHandler(apiKey, apiSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [4]:
#Sentiment Analysis
def percentage(part,whole):
    return 100 * float(part)/float(whole) 

keyword = input("Please enter keyword to search: ")
numOfTweet = int(input ("Please enter the number of tweets to analyze: "))

tweets = tweepy.Cursor(api.search_tweets, q=keyword).items(numOfTweet)
positive  = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []

for tweet in tweets:
    
    #print(tweet.text)
    tweet_list.append(tweet.text)
    analysis = TextBlob(tweet.text)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    polarity += analysis.sentiment.polarity
    
    if neg > pos:
        negative_list.append(tweet.text)
        negative += 1

    elif pos > neg:
        positive_list.append(tweet.text)
        positive += 1
    
    elif pos == neg:
        neutral_list.append(tweet.text)
        neutral += 1

positive = percentage(positive, numOfTweet)
negative = percentage(negative, numOfTweet)
neutral = percentage(neutral, numOfTweet)
polarity = percentage(polarity, numOfTweet)
positive = format(positive, '.2f')
negative = format(negative, '.2f')
neutral = format(neutral, '.2f')

Please enter keyword to search: red sox boston
Please enter the number of tweets to analyze: 100


In [5]:
#Number of Tweets (Total, Positive, Negative, Neutral)
tweet_list = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print("total number: ",len(tweet_list))
print("positive number: ",len(positive_list))
print("negative number: ", len(negative_list))
print("neutral number: ",len(neutral_list))

total number:  100
positive number:  28
negative number:  7
neutral number:  65


In [6]:
tweet_list.head(10)

,0
0,@StephenKing If memory serves that is the Bost...
1,The fact the Red Sox are even in a position to...
2,Check out Boston Red Sox Kevin Youkilis #20 ML...
3,RT @SassTreasures: Check out this listing I ju...
4,RT @JeffreyGuterman: On this date September 25...
5,Check out this listing I just added to my #Pos...
6,RT @JeffreyGuterman: On this date September 25...
7,RT @jackharlow: Boutta walk into Fenway Park f...
8,RT @YESNetwork: The Yankees defeated the Red S...
9,Yankees 7 remaining games:\nTonight's 1 game i...


In [7]:
#Calculating Negative, Positive, Neutral and Compound values

tweet_list[['polarity', 'subjectivity']] = tweet_list[0].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tweet_list[0].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
        tweet_list.loc[index, 'sentiment'] = "negative"
    elif pos > neg:
        tweet_list.loc[index, 'sentiment'] = "positive"
    else:
        tweet_list.loc[index, 'sentiment'] = "neutral"
    tweet_list.loc[index, 'neg'] = neg
    tweet_list.loc[index, 'neu'] = neu
    tweet_list.loc[index, 'pos'] = pos
    tweet_list.loc[index, 'compound'] = comp

tweet_list.head(10)

,0,polarity,subjectivity,sentiment,neg,neu,pos,compound
0,@StephenKing If memory serves that is the Bost...,-0.10000,0.300000,negative,0.184,0.816,0.000,-0.5574
1,The fact the Red Sox are even in a position to...,-0.15625,0.566667,negative,0.226,0.676,0.098,-0.5859
2,Check out Boston Red Sox Kevin Youkilis #20 ML...,0.25000,0.375000,neutral,0.000,1.000,0.000,0.0000
3,RT @SassTreasures: Check out this listing I ju...,0.25000,0.375000,neutral,0.000,1.000,0.000,0.0000
4,RT @JeffreyGuterman: On this date September 25...,0.06250,0.500000,neutral,0.000,1.000,0.000,0.0000
5,Check out this listing I just added to my #Pos...,0.25000,0.375000,neutral,0.000,1.000,0.000,0.0000
6,RT @JeffreyGuterman: On this date September 25...,0.06250,0.500000,neutral,0.000,1.000,0.000,0.0000
7,RT @jackharlow: Boutta walk into Fenway Park f...,0.00000,0.000000,neutral,0.000,1.000,0.000,0.0000
8,RT @YESNetwork: The Yankees defeated the Red S...,0.25000,0.500000,negative,0.222,0.667,0.111,-0.4574
9,Yankees 7 remaining games:\nTonight's 1 game i...,-0.20000,0.200000,neutral,0.000,1.000,0.000,0.0000


In [8]:
#Function for count_values_in single columns

def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

In [9]:
#Count_values for sentiment
count_values_in_column(tweet_list,"sentiment")

,Total,Percentage
neutral,65,65.0
positive,28,28.0
negative,7,7.0
